# 1. 任務
針對一個 SQL database，進行自然語言的問，產出自然語言的答。


以下我們以 Chinook 資料庫為範例（Chinook database 紀錄了關於一家線上音樂商店的資訊），詢問它的員工數，並希望它回答八人。

In [1]:
from pprint import pprint
from langchain_experimental.sql.base import SQLDatabase, SQLDatabaseChain
from langchain_setup import ChatOpenAI, tracing_v2_enabled_if_api_key_set


db = SQLDatabase.from_uri("sqlite:///../data/Chinook.db")
db_chain = SQLDatabaseChain.from_llm(ChatOpenAI(temperature=0), db)
with tracing_v2_enabled_if_api_key_set(project_name='tutorial'):
    db_chain.run("How many employees are there?")

[LangSmith URL]: https://smith.langchain.com/o/34ec837d-8405-462d-b949-fdfaebda792b/projects/p/fdcbda35-4d3a-418b-ab49-7e3205e630a6/r/3698c75f-a710-4246-8876-e0cb8efa2eec?poll=true


# 2. 機制
`SQLDatabaseChain` 底層會做以下的事情：
1. 建構 prompt，其中包含背景資訊、資料庫格式、使用者問題
2. LLM API: 產生 SQL query
3. 資料庫執行 SQL query，回傳結果
4. LLM API: 將上一次的 prompt 後面加上 SQL query 和結果，產生自然語言的回答

## 2.1 Prompt
其 Prompt 可以分成數個部分
- System: 設定角色、解釋任務內容、條列要求
- Format: 設定輸出的格式
- Context: 每次呼叫都可能不同、解決任務所需的資訊
- Question: 使用者輸入

In [2]:
print(db_chain.llm_chain.prompt.template)

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: 

接下來我們可以來看在`SQLDatabaseChain`中，如何描述 SQL tables，它用了一個很聰明的方法，就是用程式碼來敘述。我個人猜測，程式碼是ＬＬＭ看過並學習過的格式，比起自己想一個格式來描述並希望ＬＬＭ理解可能更好。另外它也提供了範例來讓ＬＬＭ了解各個 table 實際上的內容。

In [3]:
print(db_chain.database.get_table_info())


CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Empl

**想想看: 例如以顧客基因的資料字典來說，欄位有四百多個，其中包含欄位名`TOTAL_FC` (外幣總存款)、欄位名`DAWHO_FG` (可能值為 0 <無等級>, 1 <(大大等級>, 2 <(大戶等級>)。這樣的情況列舉欄位名稱跟舉例其值會發生什麼問題？又該如何解決呢？**

## 2.2 流程
這邊以簡化過的例子來展示實際的流程
1. 建構 prompt，其中包含背景資訊、資料庫格式、使用者問題
```
....
{table_info}
Question: How many employees are there?
```
2. LLM API: 產生 SQL query
```
...
{table_info}
Question: How many employees are there?
SQLQuery:SELECT COUNT(*) FROM Employee;
```
3. 資料庫執行 SQL query，回傳結果
```
...
{table_info}
Question: How many employees are there?
SQLQuery:SELECT COUNT(*) FROM Employee;
SQLResult: [(8,)]
```
4. LLM API: 將上一次的 prompt 後面加上 SQL query 和結果，產生自然語言的回答
```
...
{table_info}
Question: How many employees are there?
SQLQuery:SELECT COUNT(*) FROM Employee;
SQLResult: [(8,)]
Answer:There are 8 employees.
```

# 3. 資安
`SQLDatabaseChain` 也提供了一個選項 `return_direct`, 使其直接回傳 SQL Query 執行後的值，目的是讓 LLM 看不到資料庫裡的值 （只能看到 schema）。

In [4]:
db_chain = SQLDatabaseChain.from_llm(ChatOpenAI(temperature=0), db, return_direct=True, verbose=True)
db_chain.run("How many employees are there?")



> Entering new SQLDatabaseChain chain...
How many employees are there?
SQLQuery:SELECT COUNT(*) FROM Employee
SQLResult: [(8,)]
> Finished chain.


'[(8,)]'

**想想看：如果看 LLM: SQL schema -> SQL query 是雲端LLM， LLM: SQL query result -> text answer 是地端的話？**